# Step 5: Post-Scoring Checks
Verify every question has marks, validate IDs against the Name List, and clean versioned mark/control files before packaging.

In [1]:
from grading_utils import setup_paths, create_directories
from termcolor import colored
import pandas as pd
import os
import json

# Configure dataset prefix and data root used during scoring
prefix = "VTC Test"  # change if you used another exam name
dataset = "sample"   # change to your dataset folder name

# Resolve paths to generated outputs
paths = setup_paths(prefix, dataset)
base_path_questions = paths["base_path_questions"]
name_list_file = paths["name_list_file"]

# Ensure required directories exist
create_directories(paths)

# Load Name List for ID validation
try:
    name_list_df = pd.read_excel(name_list_file, sheet_name="Name List")
    print(f"✓ Loaded Name List from {name_list_file}")
except Exception as e:
    print(f"⚠️ Failed to load Name List: {e}")
    name_list_df = pd.DataFrame()

✓ Loaded Name List from ../sample/VTC Test Name List.xlsx


# Post Processing after scoring
1. Check all question has scores.
2. Check ID again.
3. Remove version history.

In [2]:
# check each sub folder of base_path_questions contains file name mark.json, ignore the root folder
import os
import json

unfinsihed_scoring = []
for path, currentDirectory, files in os.walk(base_path_questions):
    if path != base_path_questions:
        # extract question name from path
        question = path[len(base_path_questions) + 1 :]
        if "mark.json" not in files:
            unfinsihed_scoring.append(question)
        else:
            # read mark.json as json
            with open(os.path.join(path, "mark.json"), "r") as f:
                marks = json.load(f)            
            # check each mark in marks that attribute "mark" or "overridedMark" is not empty
            for mark in marks:
                if mark['mark'] == "" and  mark['overridedMark'] == "":
                    # extract question name from path                   
                    unfinsihed_scoring.append(question)
                    break             

if len(unfinsihed_scoring) > 0:            
    print(colored("{} have some question not yet mark!".format(unfinsihed_scoring), "red"))          
else:
    print("All questions have been marked!")

All questions have been marked!


Check ID

In [3]:
import os
import json
from termcolor import colored

with open(os.path.join(base_path_questions,"ID", "mark.json"), "r") as f:
    marks = json.load(f)

id_from_mark = list(map(lambda x: x["overridedMark"] if x["overridedMark"] != "" else x["mark"], marks))
id_from_namelist = name_list_df["ID"].to_list()

# convert id_from_mark to string
id_from_mark = [str(id) for id in id_from_mark]
id_from_namelist = [str(id) for id in id_from_namelist]

mark_missing_id = []
for id in id_from_namelist:
    if id not in id_from_mark:   
        mark_missing_id.append(id)
print(colored("In class but not marked - {}!".format(mark_missing_id), "red"))    

marked_but_not_in_namelist = []
for id in id_from_mark:
    if id not in id_from_namelist:   
        marked_but_not_in_namelist.append(id)

print(colored("Marked ID but not in class - {}!".format(marked_but_not_in_namelist), "red"))

In class but not marked - []!
Marked ID but not in class - []!


### Remove version history
Before you backup.

In [4]:
## remove fill start with control- or mark- and end with .json in base_path_questions recursively.
import os
for path, currentDirectory, files in os.walk(base_path_questions):
    for file in files:
        if file.startswith("control-") or file.startswith("mark-"):
            os.remove(os.path.join(path, file))

### Reset everything (Danger)
Remove mark.js and control.js

In [5]:
# import os
# for path, currentDirectory, files in os.walk(base_path_questions):
    
#     for file in files:       
#         if file == "control.json" or file == "mark.json":
#             os.remove(os.path.join(path, file))